In [1]:
import matplotlib.pyplot as plt
from PIL import Image, ImageFilter, ImageChops
import numpy as np
import os
import math
import copy

In [2]:
import torch
import torch.nn as nn
from torch.utils import data
import torch.nn.functional as F
from torch.autograd import Variable

import torchvision.transforms as transforms
import torchvision.models as models

In [3]:
device = torch.device("cuda")

In [15]:
imsize = 256
mean = torch.tensor([0.485, 0.456, 0.406]).to(device).view(-1,1,1)
std = torch.tensor([0.229, 0.224, 0.225]).to(device).view(-1,1,1)

### Helper Functions

In [5]:
def file_to_tensor(file_path):
    global imsize
    image = Image.open(file_path)
    ten_image = transforms.Compose([transforms.Resize(imsize), transforms.ToTensor()])(image).to(device)
    ten_image.unsqueeze_(0) # adds batch size = 1
    return ten_image

def image_to_tensor(image):
    global imsize
    ten_image = transforms.Compose([transforms.Resize(imsize), transforms.ToTensor()])(image).to(device)
    ten_image.unsqueeze_(0) # adds batch size = 1
    return ten_image

def tensor_to_image(tensor):
    tensor = tensor.clone().cpu().squeeze(0)
    img = transforms.ToPILImage()(tensor)
    return img

In [6]:
def preprocess_tensor(img_tensor):
    global mean, std
    return (img_tensor - mean)/std

In [7]:
def deprocess_tensor(img_tensor):
    global mean, std
    return image * std  + mean

### Main Functions

In [8]:
vgg = models.vgg19(pretrained=True).features.to(device).eval()
module_list = list(vgg.modules())

In [9]:
def objective_function():
    pass

In [10]:
def dream(net, depth, image, iterations, lr):
    net.zero_grad()
    image_tensor = Variable(image_to_tensor(image), requires_grad=True)
    #print(image_tensor.is_leaf)
    for i in range(iterations):
        out = image_tensor
        for i in range(depth):
            out = module_list[i+1](out)
        loss = out.norm()
        loss.backward()
        image_tensor.data = image_tensor.data + lr*image_tensor.grad.data
    image = tensor_to_image(image_tensor.clone())
    return image

In [11]:
def set_octaves(net, depth, image, iterations, lr, octave_scale, num_octaves):
    if num_octaves >=1:
        use_image = image.filter(ImageFilter.GaussianBlur(2))
        if min(np.asarray(use_image.size)/octave_scale) < 1:
            size = use_image.size
        else:
            size = (use_image.size[0]//octave_scale, use_image.size[1]//octave_scale)
        use_image = use_image.resize(size)
        use_image = set_octaves(net, depth, use_image, iterations, lr, octave_scale, num_octaves-1)
        size = image.size
        use_image = use_image.resize(size)
        image = ImageChops.blend(image, use_image, 0.6)
    final_image = dream(net, depth, image, iterations, lr)
    final_image = final_image.resize(image.size)
    return final_image

In [12]:
file_path = "UpdatedSets/Set4/content.jpg"
image = Image.open(file_path)

In [16]:
fim = set_octaves(vgg, 3, image, 5, 0.3, 2, 5)

RuntimeError: cuda runtime error (2) : out of memory at /opt/conda/conda-bld/pytorch_1525909934016/work/aten/src/THC/generic/THCTensorMath.cu:35

In [ ]:
plt.imshow(fim)